<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/lab11_team03/Lab11_Equipo03_EntregaFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
59 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [35]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.sql.types import StructType, StructField, FloatType, StringType
from pyspark.sql import Row
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [36]:
# Initialize SparkSession
spark = SparkSession.builder \
            .appName("CIDwSVM") \
            .config("spark.ui.port","4040") \
            .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [37]:
# Define schema for the DataFrame
Iris_schema = StructType([
    StructField("Id", FloatType(), True),
    StructField("SepalLengthCm", FloatType(), True),
    StructField("SepalWidthCm", FloatType(), True),
    StructField("PetalLengthCm", FloatType(), True),
    StructField("PetalWidthCm", FloatType(), True),
    StructField("Species", StringType(), True)
])

In [38]:
#Aqui estamos haciendo el schema
df = spark.read.format("csv").\
    option("header", "true").\
    option("mode", "permissive").\
    option("path", "/content/drive/MyDrive/Iris_Dataset.csv").\
    schema(Iris_schema).\
    load()
df.printSchema()
df.show(n=10, truncate=False)

root
 |-- Id: float (nullable = true)
 |-- SepalLengthCm: float (nullable = true)
 |-- SepalWidthCm: float (nullable = true)
 |-- PetalLengthCm: float (nullable = true)
 |-- PetalWidthCm: float (nullable = true)
 |-- Species: string (nullable = true)

+----+-------------+------------+-------------+------------+-----------+
|Id  |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+----+-------------+------------+-------------+------------+-----------+
|1.0 |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2.0 |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3.0 |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4.0 |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5.0 |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
|6.0 |5.4          |3.9         |1.7          |0.4         |Iris-setosa|
|7.0 |4.6          |3.4         |1.4          |0.3         |Iris-setosa|
|8.0 |5.0         

In [39]:
# Transform features into a single vector column
from pyspark.ml.feature import VectorAssembler
feature_cols = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Muestra el resultado
df.select("features", "Species").show(10, truncate=False)

+----------------------------------------------------------------------------+-----------+
|features                                                                    |Species    |
+----------------------------------------------------------------------------+-----------+
|[5.099999904632568,3.5,1.399999976158142,0.20000000298023224]               |Iris-setosa|
|[4.900000095367432,3.0,1.399999976158142,0.20000000298023224]               |Iris-setosa|
|[4.699999809265137,3.200000047683716,1.2999999523162842,0.20000000298023224]|Iris-setosa|
|[4.599999904632568,3.0999999046325684,1.5,0.20000000298023224]              |Iris-setosa|
|[5.0,3.5999999046325684,1.399999976158142,0.20000000298023224]              |Iris-setosa|
|[5.400000095367432,3.9000000953674316,1.7000000476837158,0.4000000059604645]|Iris-setosa|
|[4.599999904632568,3.4000000953674316,1.399999976158142,0.30000001192092896]|Iris-setosa|
|[5.0,3.4000000953674316,1.5,0.20000000298023224]                            |Iris-setosa|

In [40]:
from pyspark.ml.feature import StringIndexer
label_indexer = StringIndexer(inputCol="Species", outputCol="label")
df = label_indexer.fit(df).transform(df)


In [41]:
# Initialize the LinearSVC classifier for binary classification
lsvc = LinearSVC(maxIter=10, regParam=0.01)

# Set up OneVsRest classifier for multi-class classification
ovr = OneVsRest(classifier=lsvc)

# Train the model
ovr_model = ovr.fit(df)

# Make predictions on the test set
predictions = ovr_model.transform(df)

# Show predictions
predictions.show()

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(metricName="f1")
f1 = evaluator.evaluate(predictions)
print("Test F1 Score = ", f1)

# Stop the Spark session
spark.stop()

+----+-------------+------------+-------------+------------+-----------+--------------------+-----+--------------------+----------+
|  Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|            features|label|       rawPrediction|prediction|
+----+-------------+------------+-------------+------------+-----------+--------------------+-----+--------------------+----------+
| 1.0|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[5.09999990463256...|  0.0|[2.08191232093384...|       0.0|
| 2.0|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[4.90000009536743...|  0.0|[1.47026651060883...|       0.0|
| 3.0|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[4.69999980926513...|  0.0|[1.87015643341117...|       0.0|
| 4.0|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.59999990463256...|  0.0|[1.68565838961183...|       0.0|
| 5.0|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0